In [ ]:
import cv2
import numpy as np
import time
import os
import HandTrackingModule as htm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import models
from collections import Counter
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import wmi

In [16]:
folderPath = "HeaderForProject"
myList = os.listdir(folderPath)
overLayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    image = cv2.resize(image,(640,70))
    overLayList.append(image)

header = overLayList[0]
drawColor = (0,0,0)
brushThickness = 15
eraserThickness = 50

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

detector = htm.handDetector(detectionCon=0.85)
xp,yp = 0,0
imageCanvas = np.zeros((480,640,3),np.uint8)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volRange = volume.GetVolumeRange()
minVol = volRange[0]
maxVol = volRange[1]
volbar = 400
volpercentage = 0

c = wmi.WMI(namespace='wmi')
methods = c.WmiMonitorBrightnessMethods()[0]

while True:
    #1. importing image
    success, img = cap.read()
    img = cv2.flip(img, 1)

    #2. find hand landmarks
    img = detector.findHands(img)
    landmarkList = detector.findPosition(img,draw = False)

    vol = 0
    brightbar=150
    brightpercentage = 0    

    if len(landmarkList) != 0:

        #tip of index and middle fingers
        x1,y1 = landmarkList[8][1:]
        x2,y2 = landmarkList[12][1:]
        x3,y3 = landmarkList[4][1:]

        cx1,cy1 = (x1+x3)//2 , (y1+y3)//2

        #volume n brightness
        a=0
        length = math.hypot(x2-x1,y2-y1)


    #3. checking which fingers are up
        fingers = detector.fingersUp()

        #4. if selection mode - two fingers are up
        if fingers[1] and fingers[2] : 
            xp , yp = 0,0

            if y1 < 70 :
                if 585<x1<640:
                    header = overLayList[1]
                    drawColor=(255,255,255)
                if 525<x1<580:
                    header = overLayList[2]
                    drawColor=(0,0,255)
                if 465<x1<520:
                    header = overLayList[3]
                    drawColor =(255,0,0)
                if 405<x1<460:
                    header = overLayList[4]
                    drawColor=(0,255,0)
                if 330<x1<395:
                    header = overLayList[5]
                    drawColor=(0,0,0)
                if 165<x1<300:
                    #prediction here
                    header = overLayList[6]
                    drawColor=(0,0,0)
                    break
                if 70<x1<135:
                    #volume control
                    header = overLayList[7]
                    a=1
                    drawColor=(0,0,0)
                if 0<x1<65:
                    #brightness control
                    header = overLayList[8]
                    drawColor=(0,0,0)                                                            
                    a=2

            cv2.rectangle(img,(x1,y1-25),(x2,y2+25),drawColor,cv2.FILLED)

        #5. if drawing mode - index finger is up
        if fingers[1] and fingers[2]==0:
            cv2.circle(img,(x1,y1),15,drawColor,cv2.FILLED)

            if xp==0 and yp==0:
                xp,yp = x1,y1

            if drawColor==(0,0,0):
                cv2.line(img,(xp,yp),(x1,y1),drawColor,eraserThickness)
                cv2.line(imageCanvas,(xp,yp),(x1,y1),drawColor,eraserThickness)
            else:
                cv2.line(img,(xp,yp),(x1,y1),drawColor,brushThickness)
                cv2.line(imageCanvas,(xp,yp),(x1,y1),drawColor,brushThickness)
            
            xp,yp = x1,y1 


        if a==1:
            vol = np.interp(length,[50,200],[minVol,maxVol])
            volume.SetMasterVolumeLevel(vol, None)
        if a==2:
            brightness = int(length) 
            try:
                c.WmiMonitorBrightnessMethods()[0].WmiSetBrightness(brightness,0)  # Set brightness
            except Exception as e:
                print(f"Error settingbrightness: {e}")         
  

    imgGray = cv2.cvtColor(imageCanvas,cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray,50,255,cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img,imgInv)
    img = cv2.bitwise_or(img,imageCanvas)


# header image
    img[0:70,0:640] = header

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()

cv2.destroyAllWindows()

In [ ]:
plt.imshow(imageCanvas)

In [ ]:
model1 = models.load_model("modelsMade/model1.h5")
model2 = models.load_model("modelsMade/model2.h5")
model3 = models.load_model("modelsMade/model3.h5")

In [11]:
def predict(im,model):
  im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
  im=cv2.resize(im,(28,28))
  im.astype('float32')/255
  im=np.expand_dims(im,axis=0)
  result=model.predict([im])
  return np.argmax(result)

In [ ]:
m1= predict(imageCanvas,model1)
m2= predict(imageCanvas,model2)
m3= predict(imageCanvas,model3)

In [ ]:
votes = [m1,m2,m3]
vote_counts = Counter(votes)
most_common_value, count = vote_counts.most_common(1)[0]

print(f"Prediction is: {most_common_value}")